In [ ]:
from binance.client import Client
import pandas as pd

class Binance:

    def __init__(self, api_key, api_secret):
        self.client = Client(api_key, api_secret)

    def get_data(self, symbol, n, interval='1d'):
        klines = self.client.get_klines(symbol=symbol, interval=interval, limit=n)
        df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df.set_index('timestamp')
        return df

    def place_order(self, symbol, side, quantity, order_type='LIMIT', price=None):
        if order_type == 'LIMIT':
            order_params = {
                'symbol': symbol,
                'side': side,
                'type': order_type,
                'timeInForce': 'GTC',
                'quantity': quantity,
                'price': price
            }
        else:
            order_params = {
                'symbol': symbol,
                'side': side,
                'type': order_type,
                'quantity': quantity
            }
        return self.client.create_order(**order_params)

    def get_open_orders(self):
        orders = self.client.get_open_orders()
        df = pd.DataFrame(orders)
        return df

    def cancel_order(self, symbol, order_id):
        return self.client.cancel_order(symbol=symbol, orderId=order_id)


# Example usage:

# Replace 'your_api_key' and 'your_api_secret' with your actual Binance API credentials
binance = Binance(api_key='your_api_key', api_secret='your_api_secret')

# Replace 'BTCUSDT' with the desired trading pair
symbol = 'BTCUSDT'

# Example of getting historical data
historical_data = binance.get_data(symbol, n=30)
print("Historical Data:")
print(historical_data)

# Example of placing a market order
order_result = binance.place_order(symbol, side='BUY', quantity=0.001, order_type='MARKET')
print("Market Order Result:")
print(order_result)

# Example of placing a limit order
limit_order_result = binance.place_order(symbol, side='SELL', quantity=0.001, order_type='LIMIT', price=70000)
print("Limit Order Result:")
print(limit_order_result)

# Example of getting open orders
open_orders = binance.get_open_orders()
print("Open Orders:")
print(open_orders)

# Example of canceling an order
if not open_orders.empty:
    order_to_cancel = open_orders.iloc[0]['orderId']
    cancel_result = binance.cancel_order(symbol, order_to_cancel)
    print("Cancel Order Result:")
    print(cancel_result)


In [ ]:
import warnings
from datetime import datetime
import binance
warnings.filterwarnings("ignore")

order_duration = 0

def breakout(symbol):
    global order_duration
    
    # Import / Features engineering
    df = binance.get_data(symbol, 50)
    df["resistance"] = df["close"].rolling(30).max().shift(3)

    distance = 0.5/100

    # Yesterday values
    close_y = df["close"].shift(1).iloc[-1]
    resistance_y = df["resistance"].shift(1).iloc[-1]

    # Today values
    close = df["close"].iloc[-1]
    resistance = df["resistance"].iloc[-1]

    # Signal
    buy = (close_y < resistance_y) and (close > resistance*(1+distance))
    sell = False
    
    # Position modeling
    if buy:
        order_duration = 7
    else:
        order_duration -= 1
        
    if order_duration > 0:
        buy = True

    return buy, sell

# Replace 'your_api_key' and 'your_api_secret' with your actual Binance API credentials
binance = Binance(api_key='your_api_key', api_secret='your_api_secret')

# True = Live Trading and False = Screener
live = True

if live:
    current_account_info = binance.client.get_account()
    print("------------------------------------------------------------------")
    print("Date: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print(f"Balance: {current_account_info['balance']} USD, \t"
          f"Equity: {current_account_info['totalAssetOfBtc']} BTC, \t"
          f"Profit: {current_account_info['totalUnrealizedProfit']} USD")
    print("------------------------------------------------------------------")

info_order = {
    "Bitcoin": ["BTCUSDT", 1.],
    "Monero": ["XMRUSDT", 0.1],
    "Ethereum": ["ETHUSDT", 0.3],
    "Ethereum Classic": ["ETCUSDT", 1.],
    "Zcash": ["ZECUSDT", 1.],
}

start = datetime.now().strftime("%H:%M:%S")
while True:
    # Verification for launch
    if datetime.now().weekday() not in (5, 6):  # If you want to trade only from Monday to Friday
        is_time = datetime.now().strftime("%H:%M:%S") == start
    else:
        is_time = False

    # Launch the algorithm
    if is_time:
        # Open the trades
        for asset in info_order.keys():
            # Initialize the inputs
            symbol = info_order[asset][0]
            lot = info_order[asset][1]

            # Create the signals
            buy, sell = breakout(symbol)

            # Run the algorithm
            if live:
                binance.run(symbol, buy, sell, lot)
            else:
                print(f"Symbol: {symbol}\t"
                      f"Buy: {buy}\t"
                      f"Sell: {sell}")